In [1]:
%use kandy(0.4.4)
%use dataframe(0.11.0)


import org.jetbrains.kotlinx.dataframe.io.readCSV
val dfRaw = DataFrame.readCSV("../gclogs/pauses.csv")


// print dataframe
dfRaw.columnsCount()

16

In [2]:
dfRaw.convert { startedAt }.toLocalDateTime(kotlinx.datetime.TimeZone.UTC)
val df = dfRaw.sortBy { startedAt }

In [3]:
df.countDistinct{type}


9

In [4]:
df.columnTypes()


[kotlin.String, kotlin.String, kotlinx.datetime.Instant, kotlin.Double, kotlin.String, kotlin.Double, kotlin.Double, kotlin.Double, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int]

In [5]:
df.head(5)


DataFrame: rowsCount = 5, columnsCount = 16

In [6]:
df.maxBy{duration}


DataRow: index = 13510, columnsCount = 16

In [7]:
df.groupBy{type}.sum()


DataFrame: rowsCount = 9, columnsCount = 13

In [8]:
df.filter { type == "Young" }.sortByDesc{duration}.head(5)


DataFrame: rowsCount = 5, columnsCount = 16

In [9]:
df.schema()


type: String
event_type: String
startedAt: kotlinx.datetime.Instant
duration: Double
cause: String
cpu_user: Double
cpu_kernel: Double
cpu_wallClock: Double
heap_occupancy_before_coll: Int
heap_occupancy_after_coll: Int
heap_size_before_coll: Int
heap_size_after_coll: Int
perm_or_meta_occupancy_before_coll: Int
perm_or_meta_occupancy_after_coll: Int
perm_or_meta_size_before_coll: Int
perm_or_meta_size_after_coll: Int

In [10]:
df.groupBy{cause}.count()


DataFrame: rowsCount = 5, columnsCount = 2

In [11]:
df.describe()


DataFrame: rowsCount = 16, columnsCount = 12

In [12]:
df.select {type and duration}.groupBy{type}.sum()
        .add("max [s]"){df.filter { row -> it.type==row.type }.select("duration").max()}
        .add("mean [s]"){df.filter { row -> it.type==row.type }.select("duration").mean()}



DataFrame: rowsCount = 9, columnsCount = 4

In [13]:
df.select {type and duration}.groupBy{type}.aggregate {
    count() into "count"
    sum(duration) into "sum [s]"
    max(duration) into "max [s]"
    min(duration) into "min [s]"
    mean(duration) into "avg [s]"
 }


DataFrame: rowsCount = 9, columnsCount = 6

In [14]:
df.sum {duration}


97.10613200000051

In [15]:
df.select {event_type and duration}.groupBy{event_type}.sum()


DataFrame: rowsCount = 2, columnsCount = 2

In [16]:
df.select {type and duration}.groupBy{type}.sum()


DataFrame: rowsCount = 9, columnsCount = 2

In [17]:
df.filter { type=="Young" }.select{cause}.groupBy{cause}.count()


DataFrame: rowsCount = 2, columnsCount = 2

In [18]:
import kotlinx.datetime.TimeZone

df.convert { startedAt }.toLocalDateTime(TimeZone.UTC)


... showing only top 20 of 18622 rows DataFrame: rowsCount = 18622, columnsCount = 16

In [19]:
import kotlinx.datetime.TimeZone
import kotlinx.datetime.toJavaInstant
import java.text.SimpleDateFormat
import java.time.Duration

val first = df.first().startedAt.toJavaInstant()
val last = df.last().startedAt.toJavaInstant()
val uptimeInSeconds = Duration.between(first,last).toSeconds()
val pausedDurationInSeconds:Double=df.filter {event_type=="PauseEvent"}.select{duration}.sum().get("duration") as Double

val percentage=(pausedDurationInSeconds * 100)/uptimeInSeconds

println(first)
println(last)
println(pausedDurationInSeconds)
println(percentage)


2023-07-18T16:38:34.550Z
2023-07-19T15:50:04.875Z
83.78576800000022
0.10035425559947327


In [51]:
df.valueCounts(false) { type }.plot {
    bars {
        x(type)
        y("count")
       
        fillColor("count"<Int>()) {
            scale = continuous(range = Color.YELLOW..Color.RED)
   
        borderLine.width = 0.0
    }
    }
    layout.title = "Pauses type count"
    layout.size = 950 to 500
}





Young
 
 
 
 
 
 
 
 
 G1GCYoungInitialMark
 
 
 
 
 
 
 
 
 G1GCConcurrentUndoCycle
 
 
 
 
 
 
 
 
 ConcurrentCleanupForNextMark
 
 
 
 
 
 
 
 
 ConcurrentClearClaimedMarks
 
 
 
 
 
 
 
 
 ConcurrentRootRegionScan
 
 
 
 
 
 
 
 
 G1GCRemark
 
 
 
 
 
 
 
 
 G1GCConcurrentCleanup
 
 
 
 
 
 
 
 
 Mixed
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 2,000
 
 
 
 
 
 
 4,000
 
 
 
 
 
 
 6,000
 
 
 
 
 
 
 8,000
 
 
 
 
 
 
 10,000
 
 
 
 
 
 
 12,000
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Pauses type count
 
 
 
 
 count
 
 
 
 
 type
 
 
 
 
 
 
 
 
 count
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2,000
 
 
 
 
 
 
 
 
 4,000
 
 
 
 
 
 
 
 
 6,000
 
 
 
 
 
 
 
 
 8,000
 
 
 
 
 
 
 
 
 10,000

In [37]:
val (w, h) = 1000 to 650

phasesCount.plot {
    pie {
        slice(count)
        fillColor(type)
        size=50.0
    }
    layout {
        size = w to h
    }
 }

-0.5
 
 
 
 
 
 
 
 
 -0.4
 
 
 
 
 
 
 
 
 -0.3
 
 
 
 
 
 
 
 
 -0.2
 
 
 
 
 
 
 
 
 -0.1
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.1
 
 
 
 
 
 
 
 
 0.2
 
 
 
 
 
 
 
 
 0.3
 
 
 
 
 
 
 
 
 0.4
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 -0.4
 
 
 
 
 
 
 -0.3
 
 
 
 
 
 
 -0.2
 
 
 
 
 
 
 -0.1
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 0.1
 
 
 
 
 
 
 0.2
 
 
 
 
 
 
 0.3
 
 
 
 
 
 
 0.4
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y
 
 
 
 
 x
 
 
 
 
 
 
 
 
 type
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Young
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 G1GCYoungInitialMark
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 G1GCConcurrentUndoCycle
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ConcurrentCleanupForNextMark
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ConcurrentClearClaimedMarks
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ConcurrentRootRegionScan
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 G1GCRemark
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 G1GCConcurrentCleanup
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Mixed